# Wrangle OpenStreetMap Data

## Area 

Memphis, Tennessee

https://overpass-api.de/api/map?bbox=-90.0783,35.0688,-89.8654,35.1968

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import csv
import codecs
import sqlite3
from collections import defaultdict
import cerberus
import schema
import os

# Take a sample of the data

In [2]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "map.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# Find and count all the top level tags.

In [3]:
#Find top level tags in the data set and count them, code taken from case study. 

tags = {}

def count_tags(filename):
    for event, elem in ET.iterparse(filename ,events=("start",)):
        if elem.tag in tags.keys(): #Check if the tag is already in tags
            tags[elem.tag] += 1     #Add 1 to the count if it is. 
        else:
            tags[elem.tag] = 1      #If it isnt, set the count to 1
    return tags

with open('map.osm', 'r') as mapfile:
    pprint.pprint(count_tags(mapfile))

{'bounds': 1,
 'member': 22448,
 'meta': 1,
 'nd': 1464119,
 'node': 1331083,
 'note': 1,
 'osm': 1,
 'relation': 903,
 'tag': 218854,
 'way': 118315}


# Check K values for problem characters

In [4]:
#Code used in Case Study
#Checking the k values for each tag for problem characters

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def check_k(element, keys):
    if element.tag == "tag":
        if lower.match(element.attrib['k']):
            keys["lower"] += 1
        elif lower_colon.search(element.attrib['k']):
            keys["lower_colon"] +=1
        elif problemchars.search(element.attrib['k']):
            keys["problemchars"] += 1
        else:
            keys ["other"] += 1
    pass
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = check_k(element, keys)

    return keys

with open('map.osm', 'r') as mapfile:
    keys = process_map(mapfile)
    pprint.pprint(keys)

{'lower': 163145, 'lower_colon': 53209, 'other': 2500, 'problemchars': 0}


# Audit and correct street names.

In [5]:
#Auditing street Names and correcting any issues. Used in Case Study

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Freeway","Circle","Strand","Sterling","Way","Highway",
            "Terrace","South","East","West","North", "Cove"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s:s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit():
    with open ('map.osm', 'r') as mapfile:
        street_types = defaultdict(set)
        for event, elem in ET.iterparse(mapfile, events=("start", )):
            if elem.tag == "way" or elem.tag == "node":
                for tag in elem.iter("tag"):
                    if is_street_name(tag):
                        audit_street_type(street_types, tag.attrib['v'])
        return street_types
        
audit()

#Correcting Street Types

mapping = {
            " St ": " Street ",
            " St.": " Street ",
            " Rd.": " Road ",
            " Rd ": " Road ",
            " Rd": " Road ",
            " Ave ": " Avenue ", 
            " Ave.": " Avenue ",
            " Av ": " Avenue ",
            " Ave" : " Avenue ",
            " Dr ": " Drive ",
            " Dr.": " Drive",
            " Blvd ": " Boulevard ",
            " Blvd": " Boulevard",
            " Blvd.": " Boulevard",
            " Ct ": " Centre ",
            " Ctr": " Centre",
            " Pl ": " Place ",
            " Ln ": " Lane ",
            " Cir ": " Circle ",
            " Cir" : " Circle ",
            " Wy": " Way ",
            " S ": " South ",
            " E ": " East ",
            " W ": " West ",
            " N ": "North"
}

#Update function that will also be used when preparing the data for SQL. 
def update_name(name, mapping):
    for key, value in mapping.iteritems():
        if key in name:
            return name.replace(key, value)
    return name

with open ('map.osm', 'r') as mapfile:
    s_types = audit()
    
    for s_type, ways in s_types.iteritems():
        for name in ways:
            correct_name = update_name(name, mapping)
            print name, "->", correct_name

Covington Pike -> Covington Pike
Mississippi -> Mississippi
Perkins Extended -> Perkins Extended
Jackson -> Jackson
Front -> Front
Avon Rd -> Avon Road 
E Brookhaven Cir -> E Brookhaven Circle 
Poplar -> Poplar
Ridge Lake Blvd -> Ridge Lake Boulevard
B.B. King Blvd -> B.B. King Boulevard
Lamar Ave -> Lamar Avenue 
Shadyac Ave -> Shadyac Avenue 
W G E Patterson Ave -> W G East Patterson Ave
Chelsea Ave -> Chelsea Avenue 
Central Ave -> Central Avenue 
Lynnfield Road Suite 236 -> Lynnfield Road Suite 236
Main -> Main
Clarke Rd. -> Clarke Road .


It looks like abbreviations are the biggest issue with the street names. I'll use the update_name function when I prepare the data for SQL to correct the abbreviations. 

# Audit and correct zip codes

In [10]:
#Auditing zip codes and correcting errors, this is an alteration of the case study script for auditing street names. 

zip_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
zip_types = defaultdict(set)
expected = []

def audit_zip_code(zip_types, zip_name):
    m = zip_type_re.search(zip_name)
    if m:
        zip_type = m.group()
        if zip_type not in expected:
            zip_types[zip_type].add(zip_name)

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s:s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v)

def is_zip_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip():
    with open ('map.osm', 'r') as mapfile:
        for event, elem in ET.iterparse(mapfile, events=("start", )):
            if elem.tag == "way" or elem.tag == "node":
                for tag in elem.iter("tag"):
                    if is_zip_code(tag):
                        audit_zip_code(zip_types, tag.attrib['v'])
        return zip_types
        
def update_zipcode(zipcode): 
    if len(str(zipcode))<5:
        zipcode = 0
    return zipcode

with open ('map.osm', 'r') as mapfile:
    s_types = audit_zip()
    
    for s_type, ways in s_types.iteritems():
        for name in ways:
            correct_name = update_zipcode(name)
            print name, "->", correct_name

38111 -> 38111
38112 -> 38112
38114 -> 38114
38115 -> 38115
38117 -> 38117
38118 -> 38118
38119 -> 38119
38132 -> 38132
38134 -> 38134
3813 -> 0
3951 -> 0
38107 -> 38107
38106 -> 38106
38105 -> 38105
38104 -> 38104
38103 -> 38103
38109 -> 38109
38108 -> 38108
38128 -> 38128
38152 -> 38152
38163 -> 38163
38120 -> 38120
38122 -> 38122
38127 -> 38127
38126 -> 38126


There are two zipcodes that are incomplete. I'll convert those to 0 when preparing for SQL. 

# Preparing for database

In [12]:
#Code used in Case Study
OSM_PATH = open("map.osm", "r")

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                #This is where I will update the street abreviations and zip code errors. 
                if child.attrib['k'] == "addr:street":
                    node_tag['value'] = update_name(child.attrib['v'], mapping)
                elif child.attrib['k'] == "addr:postcode":
                    node_tag['value'] = update_zipcode(child.attrib['v'])
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']): #Check for problem characters and discard if true.
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    #This is where I will update the street abreviations and zip code errors. 
                    if child.attrib['k'] == "addr:street":
                        way_tag['value'] = update_name(child.attrib['v'], mapping)
                    elif child.attrib['k'] == "addr:postcode":
                        way_tag['value'] = update_zipcode(child.attrib['v'])
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']): #Check for problem characters and discard if true. 
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


process_map(OSM_PATH, validate=False)

# Create and connect to database

In [13]:
db = sqlite3.connect("map.db")
cur = db.cursor()

# Create tables

In [14]:
#Create Nodes
#Checking if the table exists and dropping it if it does, then create the table. 
cur.execute("DROP TABLE IF EXISTS nodes;")
db.commit()
cur.execute("CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL,lat REAL,lon REAL,user TEXT,uid INTEGER,version INTEGER,changeset INTEGER,timestamp TEXT);")
db.commit()

#Read the csv file
with open('nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

#Insert the data into the table
cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
db.commit()

#Create Nodes_Tags
cur.execute("DROP TABLE IF EXISTS nodes_tags;")
db.commit()

cur.execute("CREATE TABLE nodes_tags (id INTEGER, key TEXT, value TEXT, type TEXT, FOREIGN KEY (id) REFERENCES nodes(id))")
db.commit()

with open('nodes_tags.csv', 'rb') as f:
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
db.commit()

#Create Ways
cur.execute("DROP TABLE IF EXISTS ways;")
db.commit()

cur.execute("CREATE TABLE ways(id INTEGER PRIMARY KEY NOT NULL,user TEXT,uid INTEGER,version TEXT,changeset INTEGER,timestamp TEXT);")
db.commit()

with open('ways.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
db.commit()

#Create Ways_Nodes
cur.execute("DROP TABLE IF EXISTS ways_nodes;")
db.commit()

cur.execute("CREATE TABLE ways_nodes (id INTEGER NOT NULL,node_id INTEGER NOT NULL,position INTEGER NOT NULL,FOREIGN KEY (id) REFERENCES ways(id),FOREIGN KEY (node_id) REFERENCES nodes(id));")
db.commit()

with open('ways_nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
db.commit()

#Create Ways_Tags
cur.execute("DROP TABLE IF EXISTS ways_tags;");
db.commit()

cur.execute("CREATE TABLE ways_tags (id INTEGER NOT NULL,key TEXT NOT NULL,value TEXT NOT NULL,type TEXT,FOREIGN KEY (id) REFERENCES ways(id));")
db.commit()

with open('ways_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
db.commit()

# Queries

## File Sizes

In [15]:
print 'map.osm', '.....', (os.path.getsize("map.osm")/(1024*1024)), 'MB'
print 'sample.osm', '.....', (os.path.getsize("sample.osm")/(1024*1024)), 'MB'
print 'map.db', '.....', (os.path.getsize("map.db")/(1024*1024)), 'MB'
print 'nodes.csv', '.....', (os.path.getsize("nodes.csv")/(1024*1024)), 'MB'
print 'nodes_tags.csv', '.....', (os.path.getsize("nodes_tags.csv")/(1024*1024)), 'MB'
print 'ways.csv', '.....', (os.path.getsize("ways.csv")/(1024*1024)), 'MB'
print 'ways_tags.csv', '.....', (os.path.getsize("ways_tags.csv")/(1024*1024)), 'MB'
print 'ways_nodes.csv', '.....', (os.path.getsize("ways_nodes.csv")/(1024*1024)), 'MB'

map.osm ..... 257 MB
sample.osm ..... 26 MB
map.db ..... 127 MB
nodes.csv ..... 105 MB
nodes_tags.csv ..... 0 MB
ways.csv ..... 6 MB
ways_tags.csv ..... 6 MB
ways_nodes.csv ..... 35 MB


## Number of Nodes and Ways

In [16]:
#Number of nodes
query = cur.execute('SELECT COUNT(*) FROM nodes')
print query.fetchall()

[(1331083,)]


In [17]:
#Number of ways
query = cur.execute('SELECT COUNT(*) FROM ways')
print query.fetchall()

[(118315,)]


## Types of Nodes

In [36]:
query = cur.execute('SELECT type , COUNT(*) AS num FROM nodes_tags GROUP BY type ORDER BY num DESC;')
pprint.pprint(query.fetchall())

[(u'regular', 13225),
 (u'gnis', 4120),
 (u'addr', 968),
 (u'brand', 134),
 (u'tower', 39),
 (u'ref', 15),
 (u'contact', 15),
 (u'service', 12),
 (u'communication', 11),
 (u'social_facility', 9),
 (u'operator', 9),
 (u'historic', 9),
 (u'was', 8),
 (u'payment', 8),
 (u'traffic_signals', 4),
 (u'source', 4),
 (u'name', 4),
 (u'socket', 3),
 (u'nrhp', 3),
 (u'healthcare', 3),
 (u'fuel', 3),
 (u'toilets', 2),
 (u'removed', 2),
 (u'railway', 2),
 (u'internet_access', 2),
 (u'demolished', 2),
 (u'heritage', 1),
 (u'disused', 1),
 (u'description', 1),
 (u'census', 1)]


## Number of Unique Users

In [18]:
query = cur.execute('SELECT COUNT(distinct(uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)')
pprint.pprint(query.fetchone())

(694,)


## User with the most submissions

In [19]:
query = cur.execute('SELECT e.user, COUNT(*) AS num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) AS e GROUP BY user ORDER BY num DESC LIMIT 1;')
pprint.pprint(query.fetchall())

[(u'OSM901', 1291600)]


## Number and type of religious locations

In [20]:
query= cur.execute("SELECT value, COUNT(*) AS num FROM (SELECT key,value FROM nodes_tags UNION ALL SELECT key,value FROM ways_tags) AS e WHERE key='religion' GROUP BY value ORDER BY num DESC;")
pprint.pprint(cur.fetchall())

[(u'christian', 724),
 (u'jewish', 7),
 (u'muslim', 5),
 (u'multifaith', 1),
 (u'hindu', 1)]


## Types of Resturaunts

In [21]:
query=cur.execute("SELECT value, COUNT(*) AS num FROM (SELECT key,value FROM nodes_tags UNION ALL SELECT key,value FROM ways_tags) AS e WHERE e.key LIKE '%cuisine%' GROUP BY value ORDER BY num desc;")
pprint.pprint(cur.fetchall())

[(u'burger', 18),
 (u'american', 12),
 (u'sandwich', 8),
 (u'mexican', 8),
 (u'chicken', 8),
 (u'pizza', 7),
 (u'coffee_shop', 7),
 (u'japanese', 4),
 (u'italian', 4),
 (u'barbecue', 4),
 (u'tex-mex', 3),
 (u'seafood', 3),
 (u'ice_cream', 3),
 (u'regional', 2),
 (u'chinese', 2),
 (u'asian', 2),
 (u'wings', 1),
 (u'vietnamese', 1),
 (u'thai', 1),
 (u'steak_house', 1),
 (u'southern;breakfast', 1),
 (u'pretzel', 1),
 (u'pizza;barbecue;steak;southern;breakfast;lunch', 1),
 (u'mediterranean;korean;sandwich', 1),
 (u'gastropub', 1),
 (u'donut', 1),
 (u'diner', 1),
 (u'cookies', 1),
 (u'coffee_shop;southern', 1),
 (u'coffee;tea', 1),
 (u'chinese;sushi', 1),
 (u'chinese;buffet', 1),
 (u'cake;bagel;coffee_shop', 1),
 (u'breakfast;pancake', 1),
 (u'breakfast;coffee_shop', 1),
 (u'breakfast', 1),
 (u'bar;hotdogs', 1),
 (u'arab', 1),
 (u'american;steak', 1),
 (u'african', 1),
 (u'Club_and_Southern_Food', 1),
 (u'Bar_and_Pub_food', 1),
 (u'BBQ', 1)]


## Types of Amenities

In [32]:
query=cur.execute("SELECT value, COUNT(*) AS num FROM nodes_tags WHERE key='amenity' GROUP BY value ORDER BY num DESC;")
pprint.pprint(cur.fetchall())

[(u'place_of_worship', 688),
 (u'school', 171),
 (u'restaurant', 87),
 (u'bicycle_rental', 70),
 (u'fast_food', 22),
 (u'bar', 21),
 (u'fuel', 20),
 (u'library', 15),
 (u'cafe', 13),
 (u'post_office', 10),
 (u'social_facility', 9),
 (u'grave_yard', 8),
 (u'parking', 7),
 (u'vending_machine', 6),
 (u'toilets', 6),
 (u'theatre', 6),
 (u'pharmacy', 6),
 (u'bench', 6),
 (u'pub', 5),
 (u'kindergarten', 5),
 (u'fountain', 4),
 (u'clinic', 4),
 (u'car_rental', 4),
 (u'bicycle_repair_station', 4),
 (u'fire_station', 3),
 (u'doctors', 3),
 (u'community_centre', 3),
 (u'bank', 3),
 (u'atm', 3),
 (u'waste_basket', 2),
 (u'university', 2),
 (u'research_institute', 2),
 (u'public_building', 2),
 (u'police', 2),
 (u'nursing_home', 2),
 (u'nightclub', 2),
 (u'ice_cream', 2),
 (u'hospital', 2),
 (u'courthouse', 2),
 (u'college', 2),
 (u'charging_station', 2),
 (u'car_wash', 2),
 (u'bicycle_parking', 2),
 (u'veterinary', 1),
 (u'shelter', 1),
 (u'prison', 1),
 (u'marketplace', 1),
 (u'dentist', 1),
 (u